# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, Callable, Optional
from ecutilities.core import validate_path, validate_type, IsLocalMachine

import configparser
import numpy as np
import os
import pandas as pd
import subprocess

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



# Notebook setup

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L32){target="_blank" style="float:right; font-size:smaller"}

### nb_setup



Use in first cell of notebook to set autoreload, and paths

In [ ]:
nb_setup()

Set autoreload mode


By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def cloud_install_project_code(
    package_name:str # project package name, e.g. metagentools or git+https://github.com/repo.git@main
):
    """When nb is running in the cloud, pip install the project code package"""
    
    # test whether it runs on colab
    try:
        from google.colab import drive
        RUN_LOCALLY = False
        print('The notebook is running on colab')

    except ModuleNotFoundError:
        # not running on colab, testing is it runs on on a local machine
        RUN_LOCALLY = IsLocalMachine().is_local()
        
        if RUN_LOCALLY:
            print('The notebook is running locally, will not automatically install project code')
        else:
            print('The notebook is running on a cloud VM or the machine was not registered as local')

    if not RUN_LOCALLY:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))
        
    return RUN_LOCALLY

When using colab or another cloud VM, project code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
cloud_install_project_code(package_name='metagentools');

The notebook is running locally, will not automatically install project code


In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L81){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L89){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.126785,0.669353,2.046761,-1.003774,1.965968
1,0.589057,-1.650069,1.068545,-1.263215,-2.161413
2,0.795511,-3.000181,-0.437668,-0.625105,-2.474052


,0,1,2,3,4,5,6,7,8,9
0,0.393845,1.401846,-0.058393,0.615045,0.216824,1.018101,0.415334,-1.379481,1.072252,0.512594
1,-0.149617,0.644173,-0.766344,0.450233,-1.143174,1.752385,0.970265,-0.358164,-0.302908,1.845369
2,1.375813,0.820886,1.149883,0.152100,0.935502,-0.565842,-1.087544,0.088305,-0.510576,1.726186


In [ ]:
#| export
def df_all_cols_and_rows(
    f:Callable,   # function to apply the decorator ti
)-> Callable:     # decorated function
    """decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function"""
    @wraps(f)
    def wrapper(*args, **kwargs):
        max_rows = pd.options.display.max_rows
        max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = None
        pd.options.display.max_columns = None
        f(*args, **kwargs)
        pd.options.display.max_rows = max_rows
        pd.options.display.max_columns = max_cols
    
    return wrapper

In [ ]:
show_doc(df_all_cols_and_rows)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L96){target="_blank" style="float:right; font-size:smaller"}

### df_all_cols_and_rows

>      df_all_cols_and_rows (f:Callable)

decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| f | Callable | function to apply the decorator ti |
| **Returns** | **Callable** | **decorated function** |

Usage of the decorator

In [ ]:
@df_all_cols_and_rows
def show_df(df):
    display(df)

df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
show_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,58,89,19,18,14,95,26,77,67,49,18,74,53,28,62,57,9,92,4,72,57,42,21,71,76,42,42,57,95,69,32,24,24,76,89,60,24,22,95,44,31,41,60,3,28,29,35,17,33,20
1,32,32,56,77,61,3,26,12,13,26,24,95,76,64,89,57,42,49,27,52,6,98,10,52,40,37,76,13,21,88,88,12,79,17,7,67,34,63,2,44,50,95,11,40,47,45,80,62,87,73
2,67,54,96,16,65,61,16,26,81,67,81,48,67,40,29,10,27,92,35,66,9,98,62,78,58,45,62,10,40,31,69,67,53,78,1,94,31,90,75,22,13,88,11,95,71,37,76,35,54,43


In [ ]:
#| export
@df_all_cols_and_rows
def display_full_df(
    df:pd.DataFrame  # DataFrame to display
):
    """Display a `DataFrame` showing all rows and columns"""
#     if not isinstance(df, pd.DataFrame): raise TypeError('df must me a pandas DataFrame')
    validate_type(df, pd.DataFrame, raise_error=True)
    display(df)

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L114){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df (df:pandas.core.frame.DataFrame)

Display a `DataFrame` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame | DataFrame to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,92,85,47,36,54,21,86,43,73,3,...,19,6,57,93,12,31,53,48,37,60
1,86,3,78,48,79,44,45,29,80,2,...,64,13,76,32,99,24,51,93,76,79
2,38,94,76,47,13,18,96,84,24,24,...,91,84,92,65,42,23,67,80,35,95


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,92,85,47,36,54,21,86,43,73,3,72,49,70,42,81,27,37,61,90,30,79,54,89,50,68,65,5,89,29,84,16,85,43,8,30,86,99,83,87,65,19,6,57,93,12,31,53,48,37,60
1,86,3,78,48,79,44,45,29,80,2,96,35,34,16,26,1,56,26,45,12,96,80,26,69,58,54,46,88,43,33,42,2,28,77,78,93,17,78,28,54,64,13,76,32,99,24,51,93,76,79
2,38,94,76,47,13,18,96,84,24,24,36,0,96,29,60,25,26,38,51,86,20,22,69,18,22,37,70,1,30,33,97,15,79,18,89,76,44,24,18,27,91,84,92,65,42,23,67,80,35,95


In [ ]:
#| hide
nbdev_export()